# Assignment IV – Physics-Informed Neural Networks

Welcome to assignment 4! In this notebook we will only help with minor parts of the code and the rest is for you to fill in. If you find any errors, or have any feedback, please contact `joelwall@kth.se` or `mats.persson@mi.physics.kth.se`.

In this assignment, you will experiment with a 1D quantum mechanic problem. In particular, the aim is to see if physics information can help when learning solutions to an equation.

Your tasks are to
- Study different approaches to solving the Schrödinger equation using machine learning
- Investigate whether the combination of training data and physics information gives a better solution than either of these individually.
- Investigate if it is possible to use machine learning to solve the *inverse* problem  of reconstructing the potential from the energy levels, and if having physics information is helpful in this regard.


In [ ]:
# If you are having problems with the kernel crashing upon importing these libraries, restart the kernel and uncomment the following two lines:
# import os
# os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch.nn as nn
import torch
import torch.optim as optim
import scipy.io
import numpy as np
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load data

In [ ]:
d = scipy.io.loadmat('schrodinger_data.mat')
ood = # TODO: load out of distribution dataset
print(d.keys(), ood.keys())

## Inspect the data

In [ ]:
plt.figure()
plt.plot(d['x_inter_nm'].squeeze(), ???) # TODO: plot first training potential
plt.xlabel('x [nm]')
plt.ylabel('V(x) [eV]')
plt.title('Potential')
plt.grid(alpha=0.3)
plt.show()

# Learn spectrum from potential

In [ ]:
# --------------------
# Simple 1D CNN model
class EnergyCNN(nn.Module):
    def __init__(self, L, K):
        super().__init__()
        self.net = nn.Sequential(
            # TODO: define the CNN part of the network according to your liking. 
            nn.AdaptiveAvgPool1d(32),
            nn.Flatten(),
            nn.Linear(32*32, 128), nn.ReLU(),
            nn.Linear(128, K)
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
V_train_t = torch.tensor(d['potentials_train'], dtype=torch.float32).unsqueeze(1)
E_train_t = torch.tensor(d['sorted_eigenvalues_train'], dtype=torch.float32)

V_test_t  = torch.tensor(d['potentials_val'], dtype=torch.float32).unsqueeze(1)
E_test_t = torch.tensor(d['sorted_eigenvalues_val'], dtype=torch.float32)

L = V_train_t.shape[2]
K = E_train_t.shape[1]

model = EnergyCNN(L, K)
optimizer = ??? # define optimizer
loss_fn = nn.MSELoss()

# --------------------
# Training
t0 = time.time() # start time
for epoch in tqdm(range(300)): # try different number of epochs!
    # TODO:
    # zero gradients
    # forward pass
    # loss evaluation
    # backward pass
    # optimizer step
    
t1 = time.time() # end time

training_time = t1 - t0 # compute difference

# --------------------
# Inference on test potential
t2 = time.time()
E_pred = model(V_test_t).detach().numpy()
t3 = time.time()
inference_time = t3 - t2

print("Training time:   ", training_time)
print("Inference time:  ", inference_time)


## Visualize results

In [ ]:
def plot_spectra_results(E_pred, E_true, n_show=5):
    fig,axs=plt.subplots(1,3, figsize=(10,4), sharex=True) 
    
    axs[0].plot(E_pred[0:n_show].squeeze().T)
    axs[1].plot(E_true[0:n_show].T.cpu().numpy())
    axs[2].plot(E_pred[0:n_show].squeeze().T-E_true[0:n_show].T.cpu().numpy())
    
    plt.suptitle('Energy spectra', y=1)
    axs[0].set_title('Predicted', fontsize=9)
    axs[1].set_title('True', fontsize=9)
    axs[2].set_title('Difference', fontsize=9)
    
    axs[0].set_ylabel('Eigen energy [eV]')
    axs[1].set_xlabel('Eigen energy number')
    
    for ax in axs:
        ax.grid(alpha=0.3)

    plt.show()

plot_spectra_results(E_pred, E_test_t, n_show=15)

## Set up analytical solver

In [ ]:
hbar2DivBy2me_eVnm2 = d['hbar2DivBy2me_eVnm2'].squeeze()
deltax_nm = d['deltax_nm'].squeeze()
n_x_steps = len(d['x_nm'].squeeze())-1

def hamiltonian(V):
    # Second derivative term
    second_derivative_approximation=[1, -2, 1]/deltax_nm**2
    kin_en_matrix_first_col=-hbar2DivBy2me_eVnm2*np.concatenate([second_derivative_approximation[1:],\
        np.zeros(n_x_steps-len(second_derivative_approximation))])
    kinetic_energy_operator_eV=scipy.linalg.toeplitz(kin_en_matrix_first_col)

    # Potential term
    V_operator = np.diag(V)

    return kinetic_energy_operator_eV + V_operator

### Time comparison

In [ ]:
t0 = time.time()
for realization in tqdm(range(V_test_t.shape[0])):
    H = # TODO: get hamiltonian for the current realization of V_test_t
    E = np.linalg.eig(H)

analytical_inference_time = time.time()-t0
print(f'Analytical inference time {analytical_inference_time}')
print(f'Comparison: analytical / learned = {???:.3f}') # TODO: compute the speed up factor

## Out of distribution

In [ ]:
V_ood_t = torch.tensor(ood['potentials_ood'], dtype=torch.float32).unsqueeze(1)
E_ood_t = torch.tensor(ood['sorted_eigenvalues_ood'], dtype=torch.float32)

plt.plot(V_ood_t[0].T)
plt.title('Out-of-distribution Potential')
plt.xlabel('x [nm]')
plt.ylabel('U [eV]')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
E_pred_ood = model(V_ood_t).detach().numpy()
plot_spectra_results(E_pred_ood, E_ood_t, n_show=15)